In [1]:
#!/usr/bin/env python

import numpy as np
import cv2
from numpy.core.numeric import full
import rospy
import rospkg
import open3d as o3d
from transforms3d.quaternions import mat2quat
import time
import copy
import os
from copy import deepcopy

import sys
from os import path
from ocrtoc_common.camera_interface import CameraInterface
from ocrtoc_common.transform_interface import TransformInterface
from sensor_msgs.msg import CameraInfo, Image, PointCloud2
import pickle


if __name__ == "__main__" :
    rospack = rospkg.RosPack()
    taskid= '3-1-1-data-2'
    save_path = os.path.join(rospack.get_path('ocrtoc_perception'),'data',taskid, '')
    full_pcd = o3d.io.read_point_cloud(save_path + 'full_pcd.pcd') 
    with open(save_path +  'color_images.pickle', 'rb') as handle:
        color_images = pickle.load(handle)  
    with open(save_path + 'camera_poses.pickle', 'rb') as handle:
        camera_poses = pickle.load(handle) 
    # Compute Grasping Poses (Many Poses in a Scene)
    with open(save_path + 'gg.pickle', 'rb') as handle:
        gg = pickle.load(handle)      

    with open(save_path + 'object_poses.pickle', 'rb') as handle:
        object_poses= pickle.load(handle) 
        
        
        
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.1)        


WARNING - 2021-12-17 16:18:21,401 - rigid_transformations - autolab_core not installed as catkin package, RigidTransform ros methods will be unavailable


In [2]:
#for object_ in object_poses.values():
    #print(object_["pose"])
    
object_pose_list = list(object_poses.values())
object_pose_list_len = len(object_pose_list)

object_origin_extracted = [None]*object_pose_list_len

for i in range(object_pose_list_len):
    object_pose = object_pose_list[i]['pose']
    #object_rot = mat2quat(object_pose[0:3,0:3])
    object_trans = object_pose[0:3,3]
    
    #p_stamped.pose.position.x = object_trans[0]
    #p_stamped.pose.position.y = object_trans[1]
    #p_stamped.pose.position.z = object_trans[2]

    #p_stamped.pose.orientation.x = object_rot[0]
    #p_stamped.pose.orientation.y = object_rot[1]
    #p_stamped.pose.orientation.z = object_rot[2]
    #p_stamped.pose.orientation.w = object_rot[3]

    #p_stamped.header.frame_id = 'world'

    object_origin_extracted[i] = object_trans
    
    
print(object_origin_extracted)

[array([0.09648715, 0.00379361, 0.01701279]), array([-0.23133883, -0.07465927,  0.11760816]), array([-0.04664666, -0.24672509,  0.00993363]), array([0.09967732, 0.19631141, 0.00717068]), array([ 0.13583718, -0.06688837,  0.00704808]), array([-0.10828602,  0.14575375, -0.01189113])]


In [13]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from numpy.linalg import inv


def getTransformationsToNormals(base_offset, box_dimensions, T_w_p):

    #  Calculate distances given dimensions and offsets
    center_x = -(base_offset[0] + box_dimensions[0]/2)

    offset_x = box_dimensions[0]/2
    offset_y = box_dimensions[1]/2
    offset_z = box_dimensions[2]/2

    #  Normal translations
    N1_t = np.array([center_x + offset_x, 0        , 0])
    N2_t = np.array([center_x           , offset_y , 0])
    N3_t = np.array([center_x - offset_x, 0        , 0])
    N4_t = np.array([center_x           , -offset_y, 0])
    N5_t = np.array([center_x           , 0,  offset_z])
    N6_t = np.array([center_x           , 0, -offset_z])

    #  Normal rotations
    N1_rot = R.from_euler('z',   0, degrees=True).as_matrix()
    N2_rot = R.from_euler('z', -90, degrees=True).as_matrix()
    N3_rot = R.from_euler('z', 180, degrees=True).as_matrix()
    N4_rot = R.from_euler('z',  90, degrees=True).as_matrix()
    N5_rot = R.from_euler('x',  90, degrees=True).as_matrix()
    N6_rot = R.from_euler('x', -90, degrees=True).as_matrix()

    to_hom = np.array([0,0,0,1])

    #  Illustration of normals
    #
    #
    #                 ↑  Grasp Pose
    #        ^    
    #     offset       
    #        v        ↑  N1
    #            -----------
    #           |           |
    #           |           | 
    #      N4 ← |           | →  N2
    #           |           |
    #           |           |
    #            -----------
    #                 ↓  N3
    #  
    #   
    #  N5, N6 up and down
    

    #  Construct homogenous pose
    #  Transformation from pose to each normal
    T_p_n1 = np.vstack((np.c_[N1_rot, N1_t], to_hom))
    T_p_n2 = np.vstack((np.c_[N2_rot, N2_t], to_hom))
    T_p_n3 = np.vstack((np.c_[N3_rot, N3_t], to_hom))
    T_p_n4 = np.vstack((np.c_[N4_rot, N4_t], to_hom))
    T_p_n5 = np.vstack((np.c_[N5_rot, N5_t], to_hom))
    T_p_n6 = np.vstack((np.c_[N6_rot, N6_t], to_hom))

    #  Transformation from world to normal
    #  T_{world_to_grasp_pose} * T_{grasp_pose_to_normal}
    T_w_n1 = T_w_p*T_p_n1
    T_w_n2 = T_w_p*T_p_n2
    T_w_n3 = T_w_p*T_p_n3
    T_w_n4 = T_w_p*T_p_n4
    T_w_n5 = T_w_p*T_p_n5
    T_w_n6 = T_w_p*T_p_n6

    normals = [T_w_n1, T_w_n2, T_w_n3, T_w_n4, T_w_n5, T_w_n6]

    return normals


def pointsWithinBox(pointCloud, normals):

    tolerance = 5  # amount of points allowed within the box


    #  Mask to get points in box
    #  Initialize to all in box and change in loop
    marker = np.ones(np.shape(pointCloud)[0]).astype(int)

    print("HEJ")
    for i in range(len(normals)):
        print("in box" + str(i))
        #  Get smaller point cloud which potentially are within box

        #  Get normal pose information
        normal_pose = normals[i]
        normal_quat = R.from_matrix(normal_pose[0:3,0:3]).as_quat()
        normal_vector = np.zeros(3)
        x = normal_quat[0]; y = normal_quat[1]; z = normal_quat[2]; w = normal_quat[3]
        normal_vector[0] = 2 * (x * z - w * y)
        normal_vector[1] = 2 * (y * z + w * x)
        normal_vector[2] = 1 - 2 * (x * x + y * y)
        #normal_vector = normal_quat.apply(np.array([1,0,0]))
        print(normal_vector)
        normal_origin = normal_pose[0:3,3:4]

        
        for j in range(np.shape(pointCloud)[0]):
            point = pointCloud[j,:].reshape(3,1)

            product = np.dot(np.subtract(normal_origin,point).T, normal_vector)
            
            if np.sign(product) == -1:
                
                marker[j] = 0

    return marker

def filterPointCloud(graspList, pointCloud):

    base_offset = np.array([0.5, 0, 0]) # Base offset x, y, z
    box_dimensions = np.array([0.5, 0.5, 0.5]) #box dimensions depth, width, height

    #  Mask to discard poses
    graspMask = np.ones(len(graspList))


    for i in range(len(graspList)):
        print("Grasp " + str(i))
        #  Get grasp pose
        graspPose = graspList[i]

        #  Get normals to box
        
        normals = getTransformationsToNormals(base_offset, box_dimensions, graspPose)
        print("Normals check")
        #print(normals)
        #  If points in box, mark index as 0 to discard this pose 
        
        graspMask[i] = pointsWithinBox(pointCloud, normals)
        print("Filter check")
        print(graspMask)

    return graspMask

In [335]:
def show_points(gg, idx, pcd_full):
    
    pointCloud = np.asarray(pcd_full.points)
    
    ts = gg.translations
    rs = gg.rotation_matrices

    gg.scores = np.zeros(np.shape(gg.scores))
    gg.scores[idx] = 1
    
    pose = np.c_[rs[idx], ts[idx]]
    
    y_offset = 0.10
    z_offset = 0.03
    x_range = [-0.125, -0.025]
    y_range = [-y_offset, y_offset]
    z_range = [-z_offset, z_offset]
    
    amt_points = 10000
    
    new_points = np.zeros([amt_points,3])
    
    for i in range(amt_points):
        x = np.random.uniform(low=x_range[0], high=x_range[1])
        y = np.random.uniform(low=y_range[0], high=y_range[1])
        z = np.random.uniform(low=z_range[0], high=z_range[1])

        local_points = np.array([x,y,z,1])
        world_point = np.dot(pose, local_points)
        new_points[i,:] = world_point
           
    point_cloud_more = np.vstack((pointCloud,new_points))
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloud_more)
    o3d.visualization.draw_geometries([frame, pcd, *gg.to_open3d_geometry_list()])
    

def pointsInCollisionBox(graspRot, graspTrans, pointCloud):
    pose = np.c_[graspRot.T, -np.dot(graspRot.T,graspTrans)]
    
    y_offset = 0.10
    z_offset = 0.03
    x_range = [-0.125, -0.025]
    
    y_range = [-y_offset, y_offset]
    z_range = [-z_offset, z_offset]
    
    marker = np.zeros(np.shape(pointCloud)[0]).astype(int)
    
    for j in range(np.shape(pointCloud)[0]):
        
        point_hom = np.vstack((pointCloud[j,:].reshape(3,1),1))
        point_transformed = np.dot(pose,point_hom)
        
        #product = np.dot(np.subtract(graspTrans.reshape(3,1), point).T, normal)
        x = point_transformed[0]
        y = point_transformed[1]
        z = point_transformed[2]
        
        if x_range[0] < x < x_range[1]:
            if y_range[0] < y < y_range[1]:
                if z_range[0] < z < z_range[1]:
                    marker[j] = 1
    return marker


def filterPoses(gg, pcd):
    
    print("Filtering poses based on collision box")
    pointCloud = np.asarray(pcd.points)
    
    ts = gg.translations
    rs = gg.rotation_matrices
    
    tolerance = 100
    
    graspMask = np.ones(len(ts)).astype(int)
    
    for i in range(len(ts)):
        
        graspRot = rs[i]
        graspTrans = ts[i]
        
        marker = pointsInCollisionBox(graspRot, graspTrans, pointCloud)

                        
        print("Grasp " + str(i) + ", points in collsion box: " + str(sum(marker))) #TODO: change to print in ros

        if sum(marker) > tolerance:
            graspMask[i] = 0
        
    return graspMask
            
    
        

In [340]:
#res = filterPoses(gg, full_pcd)
print(res)
print(gg[res])

[1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.]
----------
Grasp Group, Number=17:
Grasp: score:0.0, width:0.07500000298023224, height:0.019999999552965164, depth:0.019999999552965164, translation:[-0.23096168  0.18729332  0.00830434]
rotation:
[[-0.43241397  0.08951822 -0.8972206 ]
 [-0.30086237  0.9237081   0.23716104]
 [-0.85       -0.37249163  0.3724916 ]]
object id:-1
Grasp: score:0.0, width:0.07500000298023224, height:0.019999999552965164, depth:0.009999999776482582, translation:[0.21829969 0.30593285 0.01633493]
rotation:
[[ 0.13066885  0.14096788 -0.98135304]
 [-0.17043437  0.97829807  0.11783546]
 [-0.9766667  -0.15185885 -0.15185884]]
object id:-1
Grasp: score:0.0, width:0.07500000298023224, height:0.019999999552965164, depth:0.009999999776482582, translation:[0.21829969 0.30593285 0.01633493]
rotation:
[[ 0.13066885  0.14096788 -0.98135304]
 [-0.17043437  0.97829807  0.11783546]
 [-0.9766667  -0.15185885 -0.15185884]]
object id:-1
......
Grasp: score:0.0, width:0.075000

In [321]:
#filterPoses(gg,full_pcd)
show_points(gg, 2, full_pcd)

In [322]:
idx = 0

gg.scores = np.zeros(np.shape(gg.scores))
gg.scores[idx] = 1
o3d.visualization.draw_geometries([frame, full_pcd, *gg.to_open3d_geometry_list()])

In [338]:
pointcloud = np.asarray(full_pcd.points)

idx = 11

gg.scores = np.zeros(np.shape(gg.scores))
gg.scores[idx] = 1

rot = rs[idx]

r = R.from_matrix(rot)
r_euler = r.as_euler('xyz', degrees=True)
print(rot)

#print(r_euler)
#print(gg)
marker = pointsInCollisionBox(rs[idx], ts[idx], pointcloud)
 
    
    
print(np.shape(pointcloud)[0])
print(sum(marker))
pointcloud_smaller = np.zeros((sum(marker),3))
idx = 0
for i in range(len(marker)):
    if marker[i]:
        pointcloud_smaller[idx] = pointcloud[i]
        idx = idx +1
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pointcloud_smaller)
o3d.visualization.draw_geometries([frame, pcd, *gg.to_open3d_geometry_list()])

[[-0.6265186  -0.277525   -0.728323  ]
 [-0.321519    0.9432714  -0.08285302]
 [-0.71       -0.1822607   0.68020666]]
862915
73


In [256]:
print(gg.scores)
#print(object_poses)

def assign_grasps_to_object(rs, ts, object_poses):
    dist_thresh = np.inf
    min_dists = np.inf * np.ones((len(rs)))
    min_object_ids = -1 * np.ones(shape = (len(rs)), dtype = np.int32)
    for i, object_name in enumerate(object_poses.keys()):
        #object_names.append(object_name)
        object_pose = object_poses[object_name]

        dists = np.linalg.norm(ts - object_pose['pose'][:3,3], axis=1)
        object_mask = np.logical_and(dists < min_dists, dists < dist_thresh)

        min_object_ids[object_mask] = i
        min_dists[object_mask] = dists[object_mask]
    return object_mask, min_object_ids


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [4]:

def center_of_mass_score(gg, object_poses):
    penalty_factor = 1
    ts = gg.translations
    rs = gg.rotation_matrices
    depths = gg.depths
    ts = ts + rs[:,:,0]*(np.vstack((depths, depths, depths)).T)
    dists = []
    object_mask, min_object_ids = assign_grasps_to_object(rs, ts, object_poses)
    print(min_object_ids)
    print(object_mask)
    for i, object_name in enumerate(object_poses.keys()):
        object_pose = object_poses[object_name]
        ts_temp = ts[min_object_ids == i]
        dists.append(np.linalg.norm(ts_temp-object_pose['pose'][:3,3], axis=1))
        
    dists = np.concatenate(dists)
    print(dists)
    print(gg.scores)
    gg.scores = gg.scores - dists
    gg.scores = gg.scores/max(gg.scores)
    return gg


gg = center_of_mass_score(gg, object_poses)
print(gg.scores)

[3 5 0 3 3 2 4 4 3 0 3 0 2 3 3 3 2]
[False  True False False False False False False False False False False
 False False False False False]
[0.07094071 0.04334445 0.06912329 0.0535459  0.04809061 0.06114496
 0.16133643 0.10251279 0.14279943 0.11724004 0.18946134 0.33837175
 0.10622623 0.03529682 0.00947851 0.08427956 0.13608094]
[0.04768986 0.0979858  0.19645691 0.06728294 0.12251927 0.2022797
 0.38600892 0.18790378 0.15257725 0.47451374 0.22782344 0.37251505
 0.29258    0.10572696 0.02326597 0.11828084 0.17059894]
[-0.06507854  0.15293975  0.3564036   0.03844961  0.20832391  0.39503255
  0.6288526   0.23900719  0.02736787  1.          0.10737453  0.09556623
  0.5215995   0.19713217  0.03859076  0.09516869  0.09661502]


In [5]:
o3d.visualization.draw_geometries([frame, full_pcd, *gg.to_open3d_geometry_list()])


In [ ]:
print(full_pcd.segment_plane)

dir(full_pcd)

In [48]:
out_np = np.asarray(full_pcd.points)
np.savetxt("punktmoln3-1-1.csv", out_np, delimiter = ",")